In [ ]:
%load_ext autoreload
%autoreload 2

import os,sys

code_dir = os.path.expandvars('$OSCBAGDIS_DATAPROC_CODE')
sys.path.append(code_dir)

import mne
import utils  #my code
import json
import matplotlib.pyplot as plt
import numpy as np
import h5py
import utils_preproc as upre
import gc; gc.collect()
import globvars as gv

In [ ]:
python3 run_prep_dat.py --param_file prep_dat_defparams.ini $INPUT_SUBDIR_STR


In [ ]:
mstr

In [ ]:
from os.path import join as pjoin
subdir = ''
data_dir = pjoin(gv.data_dir,subdir)

# if os.environ.get('DATA_DUSS') is not None:
#     data_dir_input = os.path.expandvars('$DATA_DUSS')   + subdir
#     data_dir_output = os.path.expandvars('$DATA_DUSS')
# else:
#     data_dir_input = '/home/demitau/data'   + subdir
#     data_dir_output = '/home/demitau/data' 
    

read_resampled = 1

    
subjinds = np.arange(1,11)
subjinds = [1,2,3,4]
subjinds = [1,2,3,4,5,7]
#subjinds = [1,3]
tasks = ['hold' , 'move', 'rest']
medstates = ['off','on']


fnames_noext = []
for subjind in subjinds:
    sis = '{:02d}'.format(subjind)
    for medstate in medstates:
        for task in tasks:
            addStr = ''

        #fnames_noext = ['S{}_off_{}'.format(sis,task), 'S{}_on_{}'.format(sis,task)]
        #fnames_noext = ['S01_off_hold', 'S01_on_hold']
        #fnames_noext = ['S01_off_move', 'S01_on_move']
        #fnames_noext = ['S02_off_move', 'S02_on_move']
            fname_noext = 'S{}_{}_{}'.format(sis,medstate,task)
            #addStr += '_resample'
            addStr += '_resample_notch_highpass'

            
#             fname = fname_noext + '.mat'
#             print('Reading {}'.format(fname) )


            if read_resampled:
                fname_full = os.path.join(data_dir,fname_noext + addStr + '_raw.fif')
            else:
                fname_full = os.path.join(data_dir,fname)
            
            #print(fname_full)
            if os.path.exists(fname_full):
                fnames_noext += [fname_noext]
                
#                 print('Warning: path does not exist!, skip! {}'.format(fname_full))
#                 continue



#rawname = rawname_ + '_resample_raw.fif'
#fname_full = os.path.join(data_dir,rawname)
    
# read file -- resampled to 256 Hz,  Electa MEG, EMG, LFP, EOG channels
#raw = mne.io.read_raw_fif(fname_full, None)

print (fnames_noext)

In [ ]:
%load_ext autoreload
%autoreload 2

rawnames_str=\
('S01_off_hold,S01_on_hold,S01_off_move,S01_on_move,'
'S02_off_hold,S02_on_hold,S02_off_move,S02_on_move,'
'S03_off_hold,S03_off_move,'
'S04_off_hold,S04_on_hold,S04_off_move,S04_on_move,'
'S05_off_hold,S05_on_hold,S05_off_move,S05_on_move,'
'S07_off_hold,S07_on_hold,S07_off_move,S07_on_move')

rawnames_str=\
('S01_off_hold,S01_on_hold,S01_off_move,S01_on_move,'
'S02_off_hold,S02_on_hold,S02_off_move,S02_on_move,'
'S03_off_hold,S03_off_move')

rawnames_str=\
('S04_off_hold,S04_on_hold,S04_off_move,S04_on_move,'
'S05_off_hold,S05_on_hold,S05_off_move,S05_on_move,'
'S07_off_hold,S07_on_hold,S07_off_move,S07_on_move')


rawnames_str = 'S05_on_move'
print(rawnames_str)


In [ ]:
from IPython import get_ipython
ipython = get_ipython()
#Once that's done, run a magic command like this:
#%debug

mstr = 'run ../run/run_prep_dat.py -r ' +   rawnames_str + \
        ' --param_file prep_dat_defparams.ini --exit_after gatherFeatStats --save_dat 0' 
ipython.magic(mstr)

In [ ]:
raws_permod_both_sides['S05_on_move']['LFP'].ch_names

In [ ]:
raws_permod_both_sides

In [ ]:
#%debug
dat_pri, dat_lfp_hires_pri, extdat_pri, anns_pri, times_pri,\
times_hires_pri, subfeature_order_pri, subfeature_order_lfp_hires_pri, aux_info_perraw = \
    ugf.collectDataFromMultiRaws(rawnames, raws_permod_both_sides, sources_type,
                             src_file_grouping_ind, src_grouping, use_main_LFP_chan,
                             'both', new_main_side, data_modalities,
                             crop_start,crop_end,msrc_inds, rec_info_pri)


In [ ]:
subfeature_order_lfp_hires_pri

In [ ]:
# reversed if needed
from utils import makeSimpleRaw
from featlist import selFeatsRegexInds
raws_permod_both_sides2 = {}
for rawi,rawn in enumerate(rawnames):
    raws_permod_both_sides2[rawn] = {}
    chis_LFP = utsne.selFeatsRegexInds(subfeature_order_pri[rawi], 'LFP.*' )
    dat_LFP = dat_pri[rawi][chis_LFP]
    raws_permod_both_sides2[rawn]['LFP'] = makeSimpleRaw(dat_LFP, np.array(subfeature_order_pri[rawi])[chis_LFP], rescale=0 )

    chis_msrc = utsne.selFeatsRegexInds(subfeature_order_pri[rawi], 'msrc.*' )
    dat_msrc = dat_pri[rawi][chis_msrc]
    raws_permod_both_sides2[rawn]['src'] = makeSimpleRaw(dat_msrc, 
                                                         np.array(subfeature_order_pri[rawi])[chis_msrc] , rescale=0)

    raws_permod_both_sides2[rawn]['LFP_hires'] = \
        makeSimpleRaw(dat_lfp_hires_pri[rawi],
                      subfeature_order_lfp_hires_pri[rawi], sfreq=sfreq_hires, rescale=0 )


In [ ]:
raws_permod_both_sides['S05_on_move']['LFP'].ch_names

In [ ]:
raws_permod_both_sides2['S05_on_move']['LFP'].ch_names

In [ ]:
raws_permod_both_sides['S05_on_move']['LFP']['LFPR01'][0]

In [ ]:
raws_permod_both_sides2['S05_on_move']['LFP']['LFPL01'][0]

In [ ]:
extdat_pri[0].shape

In [ ]:
raws_permod_both_sides2['S05_on_move'].keys()

In [ ]:
raws_permod_both_sides['S05_on_move'].keys()

## Collect stats

In [ ]:
fnames_noext = list(sorted(raws_permod_both_sides.keys() ) )

In [ ]:
modalities = ['LFP', 'src']
print(modalities)

int_types_templ = ['trem_{}', 'notrem_{}', 'hold_{}', 'move_{}']

In [ ]:
rn0 = list(raws_permod_both_sides.keys())[0]
src_chns = raws_permod_both_sides[rn0]['src'].ch_names
nsrcs_to_show = 10
src_chis = np.random.choice(len(src_chns),nsrcs_to_show)

In [ ]:
postp.printDict(raws_permod_both_sides,max_depth=1)

In [ ]:
aux_info_perraw

In [ ]:
dat_permod_perraw_perint, xshifts_rel_perint_permod = \
    upre.collecInfoForPlotHistAcrossDatasets(raws_permod_both_sides2, aux_info_perraw=aux_info_perraw,
    fnames_noext = None, side_to_use =  'main_move',int_types_templ=int_types_templ,
                                            modalities=modalities)


In [ ]:
dat_permod_perraw_perint.keys()

# Plot everything (takes time)

without rescaling, without artifact removal

In [ ]:
postp.printDict(dat_permod_perraw_perint)

In [ ]:
subfeature_order_pri[0][::90]

In [ ]:
stats_per_indset[0].keys()

In [ ]:
main_side_let = utils.getMainSide(fnames_noext[0])
int_types_to_gather_stats = [templ.format(main_side_let) for templ in int_types_templ]
print(int_types_to_gather_stats)

In [ ]:
baseline_int

In [ ]:
import utils_postprocess as postp
postp.printDict(raws_permod_both_sides,max_depth=1)

In [ ]:
indsets, means, stds = \
    upre.gatherFeatStats(rawnames, dat_T_pri, subfeature_order_pri, None, sfreq, times_pri,
            int_types_to_gather_stats, side_rev_pri = side_switched_pri,
            combine_within = combine_type, minlen_bins = 5*sfreq,
                    artif_handling=artif_handling, printLog=1, require_all_intervals_present = 0)

In [ ]:
highpass = None

#%debug
upre.plotHistAcrossDatasets(raws_permod_both_sides, dat_permod_perraw_perint, xshifts_rel_perint_permod,
                           src_chis, aux_info_perraw, highpass_used = highpass, 
                            stat_per_int=stats_per_indset[0],qsh_disp=1e-3)

In [ ]:
highpass = None

#%debug
upre.plotHistAcrossDatasets(raws_permod_both_sides, dat_permod_perraw_perint, xshifts_rel_perint_permod,
                           src_chis, aux_info_perraw, highpass_used = highpass,
                           stat_per_int=None,qsh_disp=1e-3)

In [ ]:
dat_pri[0].shape

In [ ]:
subfeature_order[0]

In [ ]:
help(utsne.selFeatsRegexInds)

In [ ]:
means[0].keys()

In [ ]:
gv.DEBUG_MODE=1

In [ ]:
baseline_int

In [ ]:
import utils_postprocess as postp

In [ ]:
means[0].keys()

In [ ]:
postp.printDict(means, print_leaves=0)

In [ ]:
np.quantile(np.zeros((2,50)), 0.5, axis=1)

In [ ]:
ipdb> bindict_per_rawn['S05_on_move']['beh_state'].keys()
dict_keys(['notrem_L', 'trem_R', 'move_L', 'undef_R', 'notrem_R', 'undef_L'])

In [ ]:
fname_full

In [ ]:
highpass = None
#highpass = 2

## On as subset

In [ ]:
#chis = utsne.selFeatsRegexInds(subfeature_order, 'LFPL*')
chis = utsne.selFeatsRegexInds(subfeature_order, 'LFP*')

subfeature_order2_pri = [ np.array(sfo)[chis] for sfo in subfeature_order_pri]
dat2_T_pri = [dat[chis].T for dat in dat_pri]

indsets, means, stds, stat_per_indset = \
        upre.gatherFeatStats(rawnames, dat2_T_pri, subfeature_order2_pri, None, sfreq, times_pri,
                int_types_to_gather_stats, side_rev_pri = side_switched_pri,
                combine_within = combine_type, minlen_bins = 5*sfreq,
                        artif_handling=artif_handling, printLog=1, require_all_intervals_present = 0)

    

In [ ]:
#stat_per_indset[0]

In [ ]:
display(means)

raws_permod_both_sides2 = {'S05_on_move':{}}
rcur = raws_permod_both_sides['S05_on_move']['LFP'] 
rcur2 = rcur.copy()

rcur2.pick_channels(subfeature_order2_pri[0])

raws_permod_both_sides2['S05_on_move']['LFP'] = rcur2

#postp.printDict(raws_permod_both_sides,max_depth=1)

display(xshifts_rel_perint_permod)

print( len( dat_permod_perraw_perint['LFP']['S05_on_move']['notrem_{}'] ) )

dat_permod_perraw_perint, xshifts_rel_perint_permod = \
    upre.collecInfoForPlotHistAcrossDatasets(raws_permod_both_sides2, aux_info_perraw=aux_info_perraw,
    fnames_noext = None, side_to_use =  'main_move',int_types_templ=int_types_templ,
                                            modalities=['LFP'])


In [ ]:
highpass = None
#%debug
upre.plotHistAcrossDatasets(raws_permod_both_sides2, dat_permod_perraw_perint, xshifts_rel_perint_permod,
                           src_chis, aux_info_perraw, highpass_used = highpass,
                           qsh_disp=1e-3, modalities=['LFP'])
upre.plotHistAcrossDatasets(raws_permod_both_sides2, dat_permod_perraw_perint, xshifts_rel_perint_permod,
                           src_chis, aux_info_perraw, highpass_used = highpass, 
                           stat_per_int = stat_per_indset[0], qsh_disp=1e-3, modalities=['LFP'])

In [ ]:
#dat_reduced = dat_pri[0][chis]
#chis = utsne.selFeatsRegexInds(subfeature_order, 'LFPL*')

## Test

In [ ]:
rawnames

In [ ]:
dat_t = np.random.uniform(low=-1,high=1,size=(2,300))
nbins_artif = 40
dat_artif_t = np.random.uniform(low=10,high=40,size=(2,nbins_artif))
dat_t[:,:nbins_artif] = dat_artif_t

In [ ]:
print( dat_t[:,0:bins_t[-1]].shape )
dat_t[0,:4] = 1e4

In [ ]:
combine_type

In [ ]:
dat_t_T_pri = [dat_t.T]
sfo_t = ['LFPL01', 'LFPR22']
times_t_pri = [np.arange(dat_t.shape[-1])/sfreq]
rawnames_t = ['S05_on_move']
bins_t = np.arange(0,int(0.5*sfreq) )
bindict_per_rawn_t = {rawnames_t[0]:{'artif':{'LFP':{'BAD_LFPL01':np.arange(nbins_artif)}, 'msrc':{}}, 
    'beh_state':{'notrem_R':bins_t } }   }

display(dat_t.mean(axis=-1))
display(utsne.robustMean(dat_t, axis=-1, per_dim = 1 ) )

display('naive interval',dat_t[:,0:bins_t[-1]+1].mean(axis=-1))
display('robust interval',utsne.robustMean(dat_t[:,0:bins_t[-1]+1], axis=-1, per_dim = 1 ) )

display('naive interval noartif',dat_t[:,nbins_artif:bins_t[-1]+1].mean(axis=-1))
display('robust interval noartif',utsne.robustMean(dat_t[:,nbins_artif:bins_t[-1]+1], axis=-1, per_dim = 1 ) )

int_types_to_gather_stats_t = ['notrem_R']

#%debug
combine_type = 'no'
side_switched_pri_t = [True]
artif_handling = 'reject'
indsets, means, stds = \
        upre.gatherFeatStats(rawnames, dat_t_T_pri, sfo_t, None, sfreq, times_t_pri,
                int_types_to_gather_stats_t, side_rev_pri = side_switched_pri_t,
                combine_within = combine_type, minlen_bins = 5,
                        artif_handling=artif_handling, printLog=1, require_all_intervals_present = 0,
                            bindict_per_rawn = bindict_per_rawn_t)




display ( indsets)
display('means', means, 'std',stds )

#########################

assert indsets == [[0]]

# total mean 
assert dat_t.mean(axis=-1)[0] > means[0]['notrem_R'][0]
# naive interval mean
assert dat_t[:,0:bins_t[-1]+1].mean(axis=-1)[0] >  means[0]['notrem_R'][0]

assert utsne.robustMean(dat_t[:,nbins_artif:bins_t[-1]+1], axis=-1, per_dim = 1)[0]  == means[0]['notrem_R'][0]

## Reading subraws

In [ ]:
# modalities = ['EMG','LFP']
# mod_picks = ['EMG.*old', 'LFP.*']

# subraws = {}
# for mod, picks in zip(modalities, mod_picks):
#     subraws[mod] = []
    
# for rawname_naked in fnames_noext: 
#     raw = uprep.getRaw(rawname_naked)
#     raw.load_data()
#     for mod, picks in zip(modalities, mod_picks):
#         subraw = uprep.getSubRaw(rawname_naked, raw = raw, picks = picks)
#         if highpass is not None:
#             subraw.filter(l_freq=2, h_freq=None, picks='all')
#         if subraw is not None:
#             subraws[mod] += [subraw]

# gc.collect()

In [ ]:
print('highpass=',highpass)

In [ ]:
fnames_noext

In [ ]:
import utils_preproc as upre

#modalities = ['EMG','LFP','MEG'] #,'msrc']
modalities = ['msrc'] #,'msrc']
rawnames = fnames_noext
raws_permod_both_sides = upre.loadRaws(rawnames,modalities, 'parcel_aal', 
                                       'parcel_ICA', 10, use_saved=False,
                                      highpass_lfreq=highpass)

gc.collect()

In [ ]:
# check channel counts agree
for mod in modalities:   
    n_channels_all = [ len(raws_permod_both_sides[rn][mod].ch_names) for rn in rawnames ] 
    unch =  np.unique(n_channels_all)
    assert len(unch ) == 1, unch


In [ ]:
sources_type = 'parcel_aal'
src_file_grouping_ind = 10
src_grouping = 0

In [ ]:
src_rec_info_fn = '{}_{}_grp{}_src_rec_info'.format(rawname_,sources_type,src_file_grouping_ind)
src_rec_info_fn_full = os.path.join(gv.data_dir, src_rec_info_fn + '.npz')
rec_info = np.load(src_rec_info_fn_full, allow_pickle=True)

roi_labels = rec_info['label_groups_dict'][()]      # dict of (orderd) lists
srcgrouping_names_sorted = rec_info['srcgroups_key_order'][()]  # order of grouping names
assert set(roi_labels.keys() ) == set(srcgrouping_names_sorted )
assert len(roi_labels) == 1, 'several groupings in single run -- not implmemented'
# assuming we have only one grouping present
roi_labels_cur = roi_labels[srcgrouping_names_sorted[src_grouping ]  ]


In [ ]:
# rescale LFP recordings at oen of the sides only according to the rest scale
if 'LFP' in modalities:
    upre.rescaleRaws(raws_permod_both_sides, mod='LFP')
if 'src' in modalities:
    upre.rescaleRaws(raws_permod_both_sides, mod='src',combine_within_medcond=True,
                    roi_labels=roi_labels, srcgrouping_names_sorted=srcgrouping_names_sorted)

In [ ]:
meg_chns = raws_permod_both_sides['S01_off_hold']['MEG'].ch_names
nmegs_to_show = 10
meg_chis = np.random.choice(len(meg_chns),nmegs_to_show)

In [ ]:
src_chns = raws_permod_both_sides['S01_off_hold']['src'].ch_names
nsrcs_to_show = 10
src_chis = np.random.choice(len(src_chns),nsrcs_to_show)

In [ ]:
vals = np.linspace(0,1, 15)
np.random.shuffle(vals)
cmap = plt.cm.colors.ListedColormap(plt.cm.jet(vals))

# Over intervals

In [ ]:
data_modalities

In [ ]:
modalities = ['LFP', 'src']
print(modalities)

int_types_templ = ['trem_{}', 'notrem_{}', 'hold_{}', 'move_{}']

In [ ]:
import globvars as gv

sfreq = raws_permod_both_sides[rawnames[0]][modalities[0]].info['sfreq']
import utils_tSNE as utsne
print(sfreq)

In [ ]:
if highpass == 2:
    szEMG = 5e-4
    szLFP = 1e-5
    defrange_permod = {'EMG':(-szEMG,szEMG), 'LFP':(-szLFP,szLFP) } # for 1 Hz fitlered data
else:
    rangeEMG = (0.00022087509714765474, 0.003293280061916448)
    rangeLFP = (-0.00016875398068805225, 0.0019851821198426474)
    defrange_permod = {'EMG':rangeEMG, 'LFP':rangeLFP}

In [ ]:
defrange_permod

In [ ]:
#defrange  = None

In [ ]:
#pp = ax.get_children()[0]

In [ ]:
cmap(vals[0])

In [ ]:
xshifts_rel_permod

In [ ]:
#timerange = 0,100
#timerange = None
nr = len(fnames_noext); nc=len(modalities)
#nr =2
ww = 10; hh = 3
fig,axs = plt.subplots(nrows=nr, ncols=nc, figsize= (nc*ww,nr*hh), sharex='col')
mns = nc*[np.inf]
mxs = nc*[-np.inf]

qsh = 5e-2
qmult = 1.15

xshifts_rel_permod = {}
for mod in modalities:
    n_channels = len( subraws[mod ][0].ch_names )
    xshifts_rel = [0]*n_channels
    xshifts_rel_permod[mod] = xshifts_rel

#xshifts0 = 0 * [nc]
    
# first gather info 
for i in range(nr):
    for j in range(nc):
        ax = axs[i,j]
        mod = modalities[j]
        raw = subraws[mod ][i]
        
        n_channels = len(raw.ch_names)
        for chni,chn in enumerate(raw.ch_names):
            chdata, times = raw[chn]
            if timerange is not None:
                sl = slice(i0,i1)
            else:
                sl = slice(None)
            chd = chdata[0,sl]
            xshift = ( np.quantile(chd,0.9)- np.quantile(chd,0.1) ) * qmult 
            
            xshifts_rel_permod[mod][chni] = max(xshifts_rel_permod[mod][chni], xshift)
        
for i in range(nr):
    for j in range(nc):     
        ax = axs[i,j]
        mod = modalities[j]
        raw = subraws[mod ][i]
        
        for chni,chn in enumerate(raw.ch_names):
            chdata, times = raw[chn]
            clr = cmap(vals[chni ])
            
            if timerange is not None:
                sl = slice(i0,i1)
            else:
                sl = slice(None)
            chd = chdata[0,sl]
            
            xshift = np.sum( xshifts_rel_permod[mod][:chni] )
            chd2 = chd + xshift
#             if defrange_permod[mod] is not None:
#                 rngshift = defrange_permod[mod] + xshift
#             else:
#                 rngshift = None
            
            q0 = np.quantile(chd2,qsh)
            q1 = np.quantile(chd2,1-qsh)
#             q0_ = q0 - np.abs(q0) * 2
#             q1_ = q1 + np.abs(q1) * 2
            q0_ = np.quantile(chd2,1e-2)
            q1_ = np.quantile(chd2,1-1e-2)
            
            ax.hist(chd2, bins=200, label=chn, alpha = 0.5, range=(q0_,q1_), color=clr )
            ax.axvline(x=np.mean(chd2), c=clr, ls=':')

            mns[j] = min(mns[j], q0)
            mxs[j] = max(mxs[j], q1)
            
            #print('{} shift {}  q = {},{}'.format(chn,xshift,q0,q1) )
            
            
        ax.legend(loc='upper left')
        ax.grid()
        ax.set_title('{}_{}_time={}'.format(fnames_noext[i], mod, timerange ) )
        
for i in range(nr):
    for j in range(nc):    
        axs[i,j].set_xlim(mns[j],mxs[j])
    
plt.savefig('across_subj_highpass{}.pdf'.format(highpass))

In [ ]:
defrange_permod[mod] 

In [ ]:
mns,mxs

In [ ]:
mns[1]*2, mxs[1]*2

In [ ]:
#help(plt.hist)

In [ ]:
np.quantile(chdata,0.98)

In [ ]:
int_types = ['trem_L', 'notrem_L', 'hold_L', 'move_L']

In [ ]:
fnames_noext

In [ ]:
ivalis_per_raw = {}
for fn in fnames_noext:
    ivalis = {'entire':[ times[0], times[-1], 'entire' ]}
    anns_fn = fn + '_anns.txt'
    anns_fn_full = os.path.join(data_dir, anns_fn)
    if os.path.exists(anns_fn_full):
        anns = mne.read_annotations(anns_fn_full)
        ivalis_ = utils.ann2ivalDict(anns)
        ivalis.update(ivalis_)
    ivalis_per_raw[fn] = ivalis

In [ ]:
for subj in gv.gen_subj_info:
    info = gv.gen_subj_info[subj]
    ts = info.get('tremor_side','UNDEF')
    ms = info.get('move_side','UNDEF')
    s = ''
    if ts != ms:
        s = '----'
    print(s,subj,ts,ms)

In [ ]:
gv.gen_subj_info['S04']['tremor_side'], gv.gen_subj_info['S04']['move_side']

In [ ]:
maintremside

In [ ]:
raw.info['sfreq']

In [ ]:
import globvars as gv
data_dir = gv.data_dir

In [ ]:
chd.shape

In [ ]:
#%debug


mn_permod = {}
mx_permod = {}
for mod in modalities:
    mn,mx = np.inf, -np.inf
    mn_permod[mod ] = mn
    mx_permod[mod ] = mx

subjinds_toshow = [1]
for subji in subjinds_toshow:  #subjinds:
    dessubj = 'S{:02d}'.format(subji)
        
    inds = []
    for ii in range(len(fnames_noext)):
        rawn = fnames_noext[ii]
        sind_str,medcond,task = utils.getParamsFromRawname(rawn)
        #print(sind_str)
        if sind_str == dessubj:
            inds += [ii]
    
    nr = len(inds); nc=len(modalities)*len(int_types)
    ww = 10; hh = 3
    fig,axs = plt.subplots(nrows=nr, ncols=nc, figsize= (nc*ww,nr*hh), sharex='col')
    plt.subplots_adjust(left=0.02,right=0.99)
    rowind = 0

    xshifts = []
    for j in range(nc):
        modind = j // len(int_types)
        
        mod = modalities[modind]
            
        for i in range(nr):
            ax = axs[i,j]
            
            fn = fnames_noext[inds[i]]
            raw = subraws[mod ][inds[i]]
#             print(i,j,fn)
#             continue
            
            ivalis = {'entire':[ times[0], times[-1], 'entire' ]}
            anns_fn = fn + '_anns.txt'
            anns_fn_full = os.path.join(data_dir, anns_fn)
            if os.path.exists(anns_fn_full):
                anns = mne.read_annotations(anns_fn_full)
                ivalis_ = utils.ann2ivalDict(anns)
                ivalis.update(ivalis_)


            int_type_cur = int_types[ j % len(int_types)  ]
            #for int_type_cur in int_types:
            for chni,chn in enumerate(raw.ch_names):
                chdata, times = raw[chn]

                #int_type_cur
                chds = []
                intervals = ivalis.get(int_type_cur, [None] )
                for timerange in intervals:
                    if timerange is not None:
                        i0, i1 = raw.time_as_index([timerange[0],timerange[1]])
                        sl = slice(i0,i1)
                    else:
                        print('{} Wrong interval type {}'.format(rawname_,int_type_cur) )
                        sl = np.arange( 2)
                    chd_ = chdata[0,sl]
                    chds += [chd_]
                chd = np.hstack(chds)
                if chd.size > 10:
                    xshift = np.quantile(chd,0.9)
                    chd2 = chd+xshift
                    
                    mn_,mx_ = np.quantile(chd2,qsh),  np.quantile(chd2,1-qsh)
                    mn_permod[mod ] = min( mn_permod[mod ], mn_ )
                    mx_permod[mod ] = max( mx_permod[mod ], mx_ )                
                    
                    clr = cmap(vals[chni ])
                    ax.hist(chd2, bins=50, label=chn, alpha = 0.6, range = defrange_permod[mod], color=clr)
                    ax.axvline(x=xshift + np.mean(chd), c=clr, ls=':')

                    print('{},{}; int_type={} min={:.3f}, max={:.3f}'.format(chn,int_type_cur,int_type_cur,mn_*10000,mx_*10000) )
            
            ax.legend(loc='upper left')
            ax.grid()
            if timerange is None:
                timerange = [-1,-1]
            ax.set_title('{}_{}_(last)time={:.2f},{:.2f}_totlen={:.2f}'.format(fn, int_type_cur, timerange[0],timerange[1], chd.size / raw.info['sfreq']  ) )
            
    for i in range(nr):
        for j in range(nc):
            modind = j // len(int_types)
            mod = modalities[modind]
            mn = mn_permod[mod ]
            mx = mx_permod[mod ]
            if not (np.isinf(mn) or np.isinf(mx) ):
                axs[i,j].set_xlim(mn,mx)

    plt.savefig('{}_highpass{}_stats_across_cond_tasks.pdf'.format(dessubj, highpass))
    plt.close()

In [ ]:
help(plt.hist)

In [ ]:
fnames_noext

In [ ]:
mn_permod

In [ ]:
mx_permod

In [ ]:
ivalis

In [ ]:
inds

In [ ]:
inds

In [ ]:
print(inds)

In [ ]:
help(plt.hist)